In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [ ]:
data = pd.read_csv("../../data/telecom_churn.csv")

In [ ]:
data.head()

In [ ]:
data.drop(["State", "Voice mail plan"], axis=1, inplace=True)

In [ ]:
data["International plan"] = data["International plan"].map({"Yes": 1, "No": 0})

In [ ]:
data.info()

In [ ]:
y = data["Churn"].astype("int")

In [ ]:
X = data.drop("Churn", axis=1)

In [ ]:
X.shape, y.shape

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.3, random_state=17
)

In [ ]:
X_train.shape, X_valid.shape

In [ ]:
first_tree = DecisionTreeClassifier(random_state=17)

In [ ]:
np.mean(cross_val_score(first_tree, X_train, y_train, cv=5))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
first_knn = KNeighborsClassifier()

In [ ]:
np.mean(cross_val_score(first_knn, X_train, y_train, cv=5))

## настраиваем max_depth для дерева

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
tree_params = {"max_depth": np.arange(1, 11), "max_features": [0.5, 0.7, 1]}

In [ ]:
tree_grid = GridSearchCV(first_tree, tree_params, cv=5, n_jobs=-1)

In [ ]:
%%time
tree_grid.fit(X_train, y_train);

In [ ]:
tree_grid.best_score_, tree_grid.best_params_

In [ ]:
knn_params = {"n_neighbors": range(5, 30, 5)}  # + list(range(50, 100, 10))}

In [ ]:
knn_grid = GridSearchCV(first_knn, knn_params, cv=5)

In [ ]:
%%time
knn_grid.fit(X_train, y_train);

In [ ]:
knn_grid.best_score_, knn_grid.best_params_

In [ ]:
tree_valid_pred = tree_grid.predict(X_valid)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_valid, tree_valid_pred)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_valid, tree_valid_pred)

In [ ]:
np.bincount(y_valid)

In [ ]:
from sklearn.tree import export_graphviz

In [ ]:
second_tree = DecisionTreeClassifier(max_depth=3).fit(X_train, y_train)
second_tree.score(X_valid, y_valid)

In [ ]:
export_graphviz(
    second_tree, out_file="telecom_tree2.dot", feature_names=X.columns, filled=True,
)

In [ ]:
!ls -l *.png

In [ ]:
!dot -Tpng telecom_tree2.dot -o telecom_tree2.png

<img src='telecom_tree2.png'>